# Support Vector Machine Learning Model (90% variance - F1 Macro)

Import required packages

In [1]:
import numpy as np # for multi-dimensional array operations
import pandas as pd # for reading data from .csv files
from sklearn.svm import SVC # for support vector machine model
from sklearn.decomposition import PCA # for principle component analysis (dimensionality reduction)
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold # for getting the best hyper parameters

Assign the training set and testing set to variables for easy reference

In [2]:
train_set = pd.read_csv('../train_tfidf_features.csv') # import the training set
test_set = pd.read_csv('../test_tfidf_features.csv') # import the testing set

Principal Component Analysis for train_set (90% variance)

In [3]:
train_set_label = train_set.loc[:, ["label"]]
features_names = [str(i) for i in range(0, 5000)]
train_set_features = train_set.loc[:, features_names] # train_set_features will not contain the label and id columns

# perform PCA
pca = PCA(n_components = 0.90)
train_set_reduced = pca.fit_transform(train_set_features)
train_set_reduced = pd.DataFrame(data = train_set_reduced)
train_set_reduced

,0,1,2,3,4,5,6,7,8,9,...,2693,2694,2695,2696,2697,2698,2699,2700,2701,2702
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,-0.006255,9.296152e-03,-0.002763,-0.009469,0.018809,0.001953,0.011959,-0.007709,0.013176,-0.000669
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,-0.006080,-6.641810e-03,0.001171,-0.003428,0.007048,0.003991,0.003874,-0.000409,0.001092,-0.005517
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,0.003468,8.876779e-03,-0.003436,-0.012670,-0.027539,-0.005050,-0.008130,-0.003846,0.024513,-0.006299
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.006023,-1.476583e-03,-0.009421,0.022844,-0.005808,0.007152,-0.000630,0.002496,0.016664,-0.003713
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,-0.012612,1.121999e-02,0.006200,0.002295,0.007828,-0.017840,0.001400,-0.002487,-0.015267,-0.001098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,0.009807,5.567380e-03,-0.004636,-0.006924,-0.005055,-0.000530,-0.003908,-0.004389,0.015175,-0.000793
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,-0.008872,8.110393e-03,-0.000660,-0.008263,0.000044,-0.008034,0.003758,-0.007950,0.010665,-0.001417
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,0.006720,-7.032342e-03,0.002390,-0.010353,-0.006662,0.014690,0.003262,0.016507,0.002151,-0.003923
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.004959,-8.777524e-07,0.004051,-0.003038,-0.001140,-0.002615,0.000768,0.006175,0.006175,-0.002205


In [4]:
X = train_set_reduced
y = train_set_label

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) # Train a SVC model using different kernel
X_train = X
y_train = y

In [5]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,2693,2694,2695,2696,2697,2698,2699,2700,2701,2702
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,-0.006255,9.296152e-03,-0.002763,-0.009469,0.018809,0.001953,0.011959,-0.007709,0.013176,-0.000669
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,-0.006080,-6.641810e-03,0.001171,-0.003428,0.007048,0.003991,0.003874,-0.000409,0.001092,-0.005517
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,0.003468,8.876779e-03,-0.003436,-0.012670,-0.027539,-0.005050,-0.008130,-0.003846,0.024513,-0.006299
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.006023,-1.476583e-03,-0.009421,0.022844,-0.005808,0.007152,-0.000630,0.002496,0.016664,-0.003713
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,-0.012612,1.121999e-02,0.006200,0.002295,0.007828,-0.017840,0.001400,-0.002487,-0.015267,-0.001098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,0.009807,5.567380e-03,-0.004636,-0.006924,-0.005055,-0.000530,-0.003908,-0.004389,0.015175,-0.000793
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,-0.008872,8.110393e-03,-0.000660,-0.008263,0.000044,-0.008034,0.003758,-0.007950,0.010665,-0.001417
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,0.006720,-7.032342e-03,0.002390,-0.010353,-0.006662,0.014690,0.003262,0.016507,0.002151,-0.003923
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.004959,-8.777524e-07,0.004051,-0.003038,-0.001140,-0.002615,0.000768,0.006175,0.006175,-0.002205


In [6]:
y_train

,label
0,1
1,0
2,1
3,0
4,1
...,...
17179,0
17180,0
17181,1
17182,1


Principal Component Analysis for test_set (90% variance)

In [7]:
features_names = [str(i) for i in range(0, 5000)]
test_set_features = test_set.loc[:, features_names] # test_set_features will not contain the label and id columns

# perform PCA
test_set_reduced = pca.transform(test_set_features) # use the pca from the train_set?
test_set_features = pd.DataFrame(data = test_set_reduced)
test_set_features

,0,1,2,3,4,5,6,7,8,9,...,2693,2694,2695,2696,2697,2698,2699,2700,2701,2702
0,-0.082801,-0.009343,-0.013543,-0.013317,-0.007984,-0.002459,-0.026278,0.004096,-0.014072,-0.020606,...,-0.006858,-0.008151,-0.008215,-0.006241,0.003272,0.007903,0.010942,0.007815,-0.010330,0.003318
1,-0.075595,0.005040,-0.017771,-0.014935,-0.000467,-0.001601,-0.055948,-0.017461,-0.031185,-0.017278,...,0.015995,0.000360,0.024518,-0.008091,0.003017,0.025033,-0.001913,-0.006084,-0.028281,0.011704
2,0.163876,0.059162,-0.057706,-0.040884,-0.063164,-0.003223,0.020473,-0.005967,-0.005539,-0.008680,...,0.004804,0.013824,0.016219,-0.003692,-0.003610,-0.003720,-0.014883,0.013688,-0.008547,-0.000307
3,-0.006835,0.189357,-0.044389,-0.024452,-0.043835,-0.013077,0.001344,0.000674,-0.005021,-0.028871,...,0.003148,-0.000298,0.004378,0.002748,-0.001563,-0.004312,0.000175,0.006652,-0.001442,0.000297
4,0.120704,-0.089485,0.015896,0.030057,-0.018015,-0.057111,-0.072387,0.053490,-0.153091,0.026020,...,-0.002217,0.001664,0.001890,-0.005852,-0.000912,-0.003947,0.004027,-0.004216,0.000969,0.003447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,-0.076808,-0.024913,-0.014752,-0.008106,-0.009817,-0.013137,0.026288,-0.012828,-0.004845,-0.010727,...,0.001878,0.002201,0.005624,0.003519,-0.001254,0.002943,-0.002659,0.006531,-0.006843,-0.005699
4292,0.069331,-0.055078,-0.008249,-0.022183,-0.000882,0.024624,-0.085988,-0.152147,0.159634,0.018128,...,0.003211,-0.000073,-0.012112,0.009394,-0.006636,-0.009583,-0.004615,-0.012078,0.006344,-0.008734
4293,0.042192,-0.056123,-0.009015,-0.022608,-0.000348,0.011589,-0.046442,-0.129357,0.126640,0.010080,...,0.007331,0.001976,-0.006988,-0.011846,-0.010437,-0.000650,-0.000948,0.006954,-0.008322,0.010148
4294,0.220417,-0.101562,-0.013420,-0.014971,-0.033857,-0.007979,-0.018445,-0.004240,-0.019587,-0.000660,...,0.002304,0.002095,-0.014773,0.004497,-0.012620,-0.012232,-0.002685,-0.003062,-0.002668,0.007388


Tuning the hyper-parameters and training the model based on the best hyper-parameters

In [8]:
# hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']} # initialise the hyper-parameters
hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} # initialise the hyper-parameters
kfold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0) # for 3-fold cross validation
# grid = GridSearchCV(SVC(), hyper_parameters, refit = True, verbose = 2) # cretae a GridSearchCV object to git to the taining data
grid = GridSearchCV(SVC(), param_grid = hyper_parameters, scoring = 'f1_macro', refit = 'f1_macro', n_jobs = 1 , cv = kfold, verbose = 2)
grid.fit(X_train, np.ravel(y_train)) # training the model using the best hyper-parameters
print(grid.best_params_) # gets the best hyper-parameters for SVM

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 5.0min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 5.0min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 5.0min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 4.5min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 4.5min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 4.5min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 4.4min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 4.5min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 4.4min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 4.3min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 4.3min
[CV] END .....................C=0.1, gamma=0.001

Predicting the labels for the test dataset based on the model with the best hyper-parameters

In [9]:
y_predicted = grid.predict(test_set_features)

In [10]:
# y_predicted = svc_model.predict(test_set_features)
y_predicted = pd.DataFrame(y_predicted, columns = ['label']) # convert y_predicted from nparray to pandas dataframe
y_predicted.insert(loc = 0, column = 'id', value = [i for i in range(17185, 17185 + 4296)]) # insert a column of the ids, starting from 17185
y_predicted.to_csv('skynet_submission_90_f1macro.csv', index = False) # output the predicted labels to ./skynet_submission_90_f1macro.csv